In [1]:
import keras
from keras import layers
from keras import initializers
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.metrics import jaccard_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence

In [2]:
#Definition of image dimensions and batch size.
IMG_HEIGHT, IMG_WIDTH = 176, 320
BATCH_SIZE = 8

In [3]:
class DataGenerator(Sequence):
    
    # Initialize the generator and sort files.
    def __init__(self, base_path, batch_size, img_size):
        self.image_folder = os.path.join(base_path, "images")  
        self.mask_folder = os.path.join(base_path, "masks")  
        self.batch_size = batch_size  
        self.img_size = img_size  
        self.image_filenames = sorted(os.listdir(self.image_folder))  
    
    # Compute the number of batches per epoch.
    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    # Get a batch of images and masks.
    def __getitem__(self, index):
        batch_files = self.image_filenames[index * self.batch_size:(index + 1) * self.batch_size]

        images, masks = [], []
        for file in batch_files:
            img_path = os.path.join(self.image_folder, file)  
            mask_path = os.path.join(self.mask_folder, file)  

            img = load_img(img_path, target_size=self.img_size)  # Load image
            mask = load_img(mask_path, target_size=self.img_size, color_mode="grayscale") 
            
            # Convert image and mask to array and normalize
            images.append(img_to_array(img) / 255.0)  
            masks.append(img_to_array(mask) / 255.0) 

        # Return batches
        return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)  

In [5]:
# Create data generators for training, validation and test datasets
train_gen = DataGenerator(os.path.join(base_dir, "train"), BATCH_SIZE, (IMG_HEIGHT, IMG_WIDTH))
val_gen = DataGenerator(os.path.join(base_dir, "val"), BATCH_SIZE, (IMG_HEIGHT, IMG_WIDTH))
test_gen = DataGenerator(os.path.join(base_dir, "test"), BATCH_SIZE, (IMG_HEIGHT, IMG_WIDTH))

In [6]:
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=1):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Output layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [7]:
# Configure early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',   # Monitor validation loss
    patience=10,          # Stop after 10 epochs without improvement
    restore_best_weights=True  # Restore model weights from the best epoch
)

# Initialize the U-Net model
model = unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=1)

# Define optimizer and compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Set the number of epochs and batch size
epochs = 100

# Train the model
history = model.fit(
    train_gen,  # Training data
    validation_data=val_gen,  # Validation data
    epochs=epochs,  # Number of training epochs
    callbacks=[early_stopping]  # Apply early stopping
)

In [9]:
# Save the trained model to a file
model.save(r"C:\Users\Mafe Valenzuela\Documents\Cps\Códigos\model.keras")